In [30]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from matplotlib import pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [31]:
model = nn.Sequential(

    # Block 1
    nn.Conv2d(3, 32, kernel_size=3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(2),      # 32×32 → 16×16

    # Block 2
    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2),      # 16×16 → 8×8

    # Block 3
    nn.Conv2d(64, 128, kernel_size=3, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(2),      # 8×8 → 4×4

    # Block 4
    nn.Conv2d(128, 256, kernel_size=3, padding=1),
    nn.BatchNorm2d(256),
    nn.ReLU(),

    nn.Flatten(),

    nn.Linear(256 * 4 * 4, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 100)   # CIFAR-100
)


In [32]:
# 3. Προετοιμασία συνόλου δεδομένων CIFAR-100

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.CIFAR100(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR100(root='./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [33]:
# 4. Ορισμός συναρτήσεων κόστους και βελτιστοποίησης
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [34]:
# --------------------------------------------------------
# 5. Εκπαίδευση
# --------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)

epochs = 30
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

cuda
Epoch 1/30, Loss: 4.5317
Epoch 2/30, Loss: 4.3358
Epoch 3/30, Loss: 4.1701
Epoch 4/30, Loss: 4.0364
Epoch 5/30, Loss: 3.9186
Epoch 6/30, Loss: 3.8123
Epoch 7/30, Loss: 3.7188
Epoch 8/30, Loss: 3.6287
Epoch 9/30, Loss: 3.5529
Epoch 10/30, Loss: 3.4714
Epoch 11/30, Loss: 3.4009
Epoch 12/30, Loss: 3.3282
Epoch 13/30, Loss: 3.2601
Epoch 14/30, Loss: 3.2015
Epoch 15/30, Loss: 3.1395
Epoch 16/30, Loss: 3.0838
Epoch 17/30, Loss: 3.0274
Epoch 18/30, Loss: 2.9734
Epoch 19/30, Loss: 2.9247
Epoch 20/30, Loss: 2.8770
Epoch 21/30, Loss: 2.8327
Epoch 22/30, Loss: 2.7907
Epoch 23/30, Loss: 2.7529
Epoch 24/30, Loss: 2.7191
Epoch 25/30, Loss: 2.6808
Epoch 26/30, Loss: 2.6473
Epoch 27/30, Loss: 2.6145
Epoch 28/30, Loss: 2.5758
Epoch 29/30, Loss: 2.5507
Epoch 30/30, Loss: 2.5244


In [35]:
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 36.11%
